imports

In [1]:
import numpy as np, sys

np.random.seed(1)

from keras.datasets import mnist



/Users/ksetdekov/Code/grokking-deep-learning/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# images, labels = (x_train[0:1000].reshape(1000,28*28) / 255,
#                   y_train[0:1000])


# one_hot_labels = np.zeros((len(labels),10))
# for i,l in enumerate(labels):
#     one_hot_labels[i][l] = 1
# labels = one_hot_labels

# test_images = x_test.reshape(len(x_test),28*28) / 255
# test_labels = np.zeros((len(y_test),10))
# for i,l in enumerate(y_test):
#     test_labels[i][l] = 1

# def tanh(x):
#     return np.tanh(x)

# def tanh2deriv(output):
#     return 1 - (output ** 2)

# def softmax(x):
#     temp = np.exp(x)
#     return temp / np.sum(temp, axis=1, keepdims=True)

# alpha, iterations = (2, 300)
# pixels_per_image, num_labels = (784, 10)
# batch_size = 128

# input_rows = 28
# input_cols = 28

# kernel_rows = 3
# kernel_cols = 3
# num_kernels = 16

# hidden_size = ((input_rows - kernel_rows) *
#                (input_cols - kernel_cols)) * num_kernels

# kernels = 0.02*np.random.random((kernel_rows*kernel_cols,
#                                  num_kernels))-0.01

# weights_1_2 = 0.2*np.random.random((hidden_size,
#                                     num_labels)) - 0.1



# def get_image_section(layer,row_from, row_to, col_from, col_to):
#     section = layer[:,row_from:row_to,col_from:col_to]
#     return section.reshape(-1,1,row_to-row_from, col_to-col_from)

# for j in range(iterations):
#     correct_cnt = 0
#     for i in range(int(len(images) / batch_size)):
#         batch_start, batch_end=((i * batch_size),((i+1)*batch_size))
#         layer_0 = images[batch_start:batch_end]
#         layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
#         layer_0.shape

#         sects = list()
#         for row_start in range(layer_0.shape[1]-kernel_rows):
#             for col_start in range(layer_0.shape[2] - kernel_cols):
#                 sect = get_image_section(layer_0,
#                                          row_start,
#                                          row_start+kernel_rows,
#                                          col_start,
#                                          col_start+kernel_cols)
#                 sects.append(sect)

#         expanded_input = np.concatenate(sects,axis=1)
#         es = expanded_input.shape
#         flattened_input = expanded_input.reshape(es[0]*es[1],-1)

#         kernel_output = flattened_input.dot(kernels)
#         layer_1 = tanh(kernel_output.reshape(es[0],-1))
#         dropout_mask = np.random.randint(2,size=layer_1.shape)
#         layer_1 *= dropout_mask * 2
#         layer_2 = softmax(np.dot(layer_1,weights_1_2))

#         for k in range(batch_size):
#             labelset = labels[batch_start+k:batch_start+k+1]
#             _inc = int(np.argmax(layer_2[k:k+1]) ==
#                                np.argmax(labelset))
#             correct_cnt += _inc

#         layer_2_delta = (labels[batch_start:batch_end]-layer_2)\
#                         / (batch_size * layer_2.shape[0])
#         layer_1_delta = layer_2_delta.dot(weights_1_2.T) * \
#                         tanh2deriv(layer_1)
#         layer_1_delta *= dropout_mask
#         weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
#         l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
#         k_update = flattened_input.T.dot(l1d_reshape)
#         kernels -= alpha * k_update

#     test_correct_cnt = 0

#     for i in range(len(test_images)):

#         layer_0 = test_images[i:i+1]
#         layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
#         layer_0.shape

#         sects = list()
#         for row_start in range(layer_0.shape[1]-kernel_rows):
#             for col_start in range(layer_0.shape[2] - kernel_cols):
#                 sect = get_image_section(layer_0,
#                                          row_start,
#                                          row_start+kernel_rows,
#                                          col_start,
#                                          col_start+kernel_cols)
#                 sects.append(sect)

#         expanded_input = np.concatenate(sects,axis=1)
#         es = expanded_input.shape
#         flattened_input = expanded_input.reshape(es[0]*es[1],-1)

#         kernel_output = flattened_input.dot(kernels)
#         layer_1 = tanh(kernel_output.reshape(es[0],-1))
#         layer_2 = np.dot(layer_1,weights_1_2)

#         test_correct_cnt += int(np.argmax(layer_2) ==
#                                 np.argmax(test_labels[i:i+1]))
#     if(j % 1 == 0):
#         sys.stdout.write("\n"+ \
#          "I:" + str(j) + \
#          " Test-Acc:"+str(test_correct_cnt/float(len(test_images)))+\
#          " Train-Acc:" + str(correct_cnt/float(len(images))))

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset
from keras.datasets import mnist

# Prepare data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = torch.tensor(x_train[:1000], dtype=torch.float32).unsqueeze(1) / 255.0
y_train = torch.tensor(y_train[:1000], dtype=torch.long)
x_test = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1) / 255.0
y_test = torch.tensor(y_test, dtype=torch.long)

train_ds = TensorDataset(x_train, y_train)
test_ds = TensorDataset(x_test, y_test)

class SimpleCNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 16, kernel_size=3)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(16 * 26 * 26, 10)
        self.save_hyperparameters()

        # Optional: Match NumPy initialization
        nn.init.uniform_(self.conv.weight, -0.01, 0.01)
        nn.init.uniform_(self.fc.weight, -0.1, 0.1)

    def forward(self, x):
        x = torch.tanh(self.conv(x))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        # Lower learning rate for stability
        return torch.optim.SGD(self.parameters(), lr=0.1)

batch_size = 128
num_workers = 7  # or use os.cpu_count() for all available cores

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers, persistent_workers=True)
val_loader = DataLoader(test_ds, batch_size=batch_size, num_workers=num_workers, persistent_workers=True)


model = SimpleCNN()
if torch.cuda.is_available():
    trainer = pl.Trainer(max_epochs=300, accelerator="auto", devices=1)
else:
    trainer = pl.Trainer(max_epochs=300, accelerator="auto")
trainer.fit(model, train_loader, val_loader)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | conv    | Conv2d  | 160    | train
1 | dropout | Dropout | 0      | train
2 | fc      | Linear  | 108 K  | train
--------------------------------------------
108 K     Trainable params
0         Non-trainable params
108 K     Total params
0.433     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Epoch 299: 100%|██████████| 8/8 [00:00<00:00, 17.93it/s, v_num=8, train_loss=0.0107, train_acc=1.000, val_loss=0.811, val_acc=0.865]  

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 8/8 [00:00<00:00, 17.74it/s, v_num=8, train_loss=0.0107, train_acc=1.000, val_loss=0.811, val_acc=0.865]


In [6]:

# After training, evaluate accuracy on train and test sets
def evaluate_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            logits = model(x)
            preds = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total

train_acc = evaluate_accuracy(model, train_loader)
test_acc = evaluate_accuracy(model, val_loader)
print(f"\nFinal Train-Acc: {train_acc:.4f} Test-Acc: {test_acc:.4f}")




Final Train-Acc: 1.0000 Test-Acc: 0.8655
